In [6]:
import pandas as pd
import countryinfo as cf # Source: https://gist.github.com/canfixit/1662664


In [7]:
countries = cf.countries
eu_codes = []
eu_names = []
iso_name = {}
exclusion_list = ['San Marino', 'Russia', 'Vatican City', 'Monaco']
for co in countries:
    
    # Excluding Russia, Monacco, San Marino and Vatican City bc. lack of data
    if co['name'] in exclusion_list:
        print(co['name'])
        continue
        
    if co['continent'].lower() == 'europe':
        iso = co['code']
        name = co['name']
        eu_codes.append(iso)
        eu_names.append(name)
        iso_name[iso] = name
        
tot_num_c = len(eu_codes)
print("Number of countries: ", tot_num_c)

Russia
San Marino
Vatican City
Monaco
Number of countries:  40


#### Source: https://gist.github.com/canfixit/1662664


#### Must rename some of the countries in some datasets in order to merge on the right values, as the source uses different names for the countries. Best method is to use ISO-codes, as these are generalistic but not all datasets might be provided with ISO, only country name. 

In [8]:
"""
Temperature x
Emissions, Population x
GDP x
Green bonds x
Environmental taxes
Environmental protection expenditure
Electric vehicles x
Greenhouse policies x
"""
# Main dataframe
df_main = None
# Time period from 2012 to 2022
time_period = list(range(2012, 2022))

In [9]:
df_temp_change = pd.read_csv("data/Annual_Surface_Temperature_Change.csv")

df_temp_change = df_temp_change[df_temp_change['ISO2'].isin(eu_codes)]

df_temp_change = df_temp_change.drop(['Unit', 'CTS_Code', 'Indicator', 'Source', 'CTS_Name', 'ObjectId', 'CTS_Full_Descriptor'], axis = 1, inplace=False)
df_temp_change.columns = [str(x).split("F")[-1] for x in df_temp_change.columns]

df_temp_change.columns= df_temp_change.columns.str.lower()
df_temp_change = pd.melt(df_temp_change, id_vars=["country", "iso2", 'iso3'], var_name = ['year'], value_name = 'temp_change C')
df_temp_change["year"] = pd.to_numeric(df_temp_change["year"])

df_temp_change.sort_values(by=["country", 'year'], inplace=True)
df_temp_change.columns = ['country', 'ISO2', 'ISO3', 'year', 'Temperature Change C']
df_temp_change.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2440 entries, 0 to 2439
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country               2440 non-null   object 
 1   ISO2                  2440 non-null   object 
 2   ISO3                  2440 non-null   object 
 3   year                  2440 non-null   int64  
 4   Temperature Change C  1898 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 114.4+ KB


In [11]:
df_co2 = pd.read_csv("data/annual-co2-emissions-per-country.csv")
df_co2.columns = df_co2.columns.str.lower()
df_co2.columns = ['country', 'ISO3', 'year', 'annual co2 emmisions']
df_co2 = df_co2[df_co2['ISO3'].isin(set(df_temp_change['ISO3']))]

df_co2["year"] = pd.to_numeric(df_co2["year"])
df_co2 = df_co2[['ISO3', 'year', 'annual co2 emmisions']]
df_co2.rename(columns={'annual co2 emmisions' : 'annual t co2 emmisions'}, inplace = True)
print("All countries: ", len((set(eu_codes) - set(df_co2['ISO3']))) == 0)
df_co2.info()

All countries:  False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6556 entries, 517 to 29726
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ISO3                    6556 non-null   object 
 1   year                    6556 non-null   int64  
 2   annual t co2 emmisions  6556 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 204.9+ KB


In [12]:
df_gdp = pd.read_csv("data/GDP.csv",delimiter=";")
#Drop last column, as it is no values
df_gdp = df_gdp.iloc[: , :-1]

df_gdp = df_gdp[df_gdp['Country Code'].isin(set(df_temp_change['ISO3']))]
df_gdp.drop(['Country Name', 'Indicator Name', 'Indicator Code'], axis = 1, inplace = True)
df_gdp.columns = df_gdp.columns.str.lower()
df_gdp.rename(columns = {'country code' : "ISO3"}, inplace=True)

df_gdp = pd.melt(df_gdp, id_vars=["ISO3"], var_name = ['year'], value_name = 'GDP US$')
df_gdp["year"] = pd.to_numeric(df_gdp["year"])

print("All countries: ", len((set(eu_codes) - set(df_gdp['ISO3']))) == 0)

df_gdp.info()

All countries:  False
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ISO3     2480 non-null   object 
 1   year     2480 non-null   int64  
 2   GDP US$  1824 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 58.2+ KB


In [14]:
# Green Bonds (in Billion $) Probably not inflation adjusted

df_green_bonds = pd.read_csv("data/Green_Bonds.csv")

df_green_bonds = df_green_bonds[df_green_bonds['ISO3'].isin(set(df_temp_change['ISO3']))]
df_green_bonds.tail(3)
df_green_bonds.drop(['ISO2', 'Indicator', 'Source', 'CTS_Code', 'CTS_Full_Descriptor', 'Type_of_Issuer', 'Use_of_Proceed', 'Principal_Currency'],axis = 1, inplace=True)
df_green_bonds.columns = df_green_bonds.columns.str.lower()
df_green_bonds.columns = [str(x).split("f")[-1] for x in df_green_bonds.columns]

# Only keep cts_name equals Green Bonds Issuances, as we can calculate total later (cts_name == Green Bonds)
df_green_bonds = df_green_bonds.loc[df_green_bonds['cts_name'] == 'Green Bonds Issuances'] 
df_green_bonds.drop(['cts_name', 'unit', 'objectid', 'country'],axis = 1, inplace=True)

df_green_bonds.rename(columns = {'iso3' : "ISO3"}, inplace=True)

df_green_bonds = pd.melt(df_green_bonds, id_vars=["ISO3"], var_name = ['year'], value_name = 'Green Bonds Issuance US$')
df_green_bonds["year"] = pd.to_numeric(df_green_bonds["year"])
df_green_bonds.sort_values(by=["ISO3", 'year'], inplace=True)

print("Missing countries: ", (set(eu_codes) - set(df_green_bonds['ISO3'])))

df_green_bonds[df_green_bonds['ISO3'] == 'GBR']

Missing countries:  {'AT', 'GB', 'LV', 'FI', 'BY', 'NO', 'MK', 'UA', 'LI', 'ME', 'DK', 'DE', 'IE', 'PT', 'SK', 'GR', 'AL', 'HR', 'BE', 'ES', 'BG', 'IT', 'HU', 'SE', 'LU', 'BA', 'IS', 'MD', 'LT', 'SI', 'AD', 'CZ', 'NL', 'RO', 'PL', 'FR', 'RS', 'EE', 'MT', 'CH'}


,ISO3,year,Green Bonds Issuance US$
29,GBR,1985,NaN
59,GBR,1986,NaN
89,GBR,1987,NaN
119,GBR,1990,NaN
149,GBR,1991,NaN
179,GBR,1992,NaN
209,GBR,1993,NaN
239,GBR,1994,NaN
269,GBR,1999,NaN
299,GBR,2000,NaN


As we observe from the output, we are missing substantial data from some countries, as well as from many years, as green bonds issuance is quite new

In [ ]:
# Population df

df_pop = pd.read_csv("data/population.csv")
df_pop = df_pop[df_pop['Country Code'].isin(set(df_temp_change_euro['ISO3']))]
df_pop.columns = df_pop.columns.str.lower()
df_pop.columns = [str(x).split("[")[0] for x in df_pop.columns]
df_pop.drop(['series name', 'series code', 'country name'] ,axis=1, inplace = True)
df_pop.rename(columns = {'country code' : "ISO3"}, inplace=True)
df_pop = pd.melt(df_pop, id_vars=["ISO3"], var_name = ['year'], value_name = 'population')
df_pop["year"] = pd.to_numeric(df_pop["year"])
df_pop["population"] = pd.to_numeric(df_pop["population"])
df_pop.info()

In [ ]:
df_sales = pd.read_csv("data/IEA-EV-dataEV salesCarsHistorical.csv")
df_ev_chargingP = pd.read_csv("data/IEA-EV-dataEV charging pointsHistorical.csv")
#df_sales_share = pd.read_csv("data/")

Notes

BEVs are battery electric vehicles. 

PHEVs are plug-in hybrid electric vehicles. 

FCEVs are fuel cell electric vehicles. 

EVs refers to all electric vehicles (BEVs + PHEVs).

In [ ]:
df_sales[df_sales['region'] == "Norway"].head()


In [ ]:

# one for BEVs and one for PHEV ??
df_bev_sales = df_sales[df_sales['powertrain'] == "BEV"]
df_phev_sales = df_sales[df_sales['powertrain'] == "PHEV"]
df_bev_sales.head()

In [ ]:
df_ev_chargingP[df_ev_chargingP['region'] == "Italy"].head()

In [ ]:
## EEA Greenhouse Policies by country

df_greenhouse_policies = pd.read_csv("data/EEA_greenhouse_policies.csv")
df_greenhouse_policies.head(1)

In [ ]:
set(eu_names) - set(df_greenhouse_policies['Country'].unique())

In [ ]:
set(df_greenhouse_policies['Country'].unique()) - set(eu_names)

In [ ]:
country_name_change = { 
    'Czechia' : 'Czech Republic', 
    'Ireland' : 'Republic of Ireland', 
    'Netherlands' : 'Kingdom of the Netherlands'}

# Merging into one main dataframe
- With average and total as two seperate datasheets
- Writing all final DataFrames into a complete Excel file

In [ ]:
df_main = df_temp_done

# Check if all conseccutive years are present
## Check if all years still in the dataset for every country
for c in df_main['country'].unique():
    for_this_c = df_main[df_main['country'] == c]
    all_years = (for_this_c['year'] == list(range(min(for_this_c['year']), max(for_this_c['year']) + 1))).all()
    if not all_years:
        print("Not Consecutive years included for: ", c)
print("Rest have all years included")

# Check if all countries in dataset
print("Number of countries before: ",len(df_main['country'].unique()) == len(eu_codes))

# Merge instead of join, because join uses index, while merge could specify
# multiple columns
# Using left joint, since we want to hold all values already in the dataframe
# for instance the years, so all missing years in later merged dataframes
# will be set to Nans

# Merge all dataframes

dfs = [df_co2, df_gdp, df_green_bonds, df_pop]

for frame in dfs:
    
    df_main = df_main.merge(frame, left_on=['ISO3','year'], right_on = ['ISO3', 'year'], how = 'left')

# Insert country name
df_main['country'] = df_main['ISO2']
df_main['country'].replace(iso_name, inplace=True)

## Only keep relevant time period
df_main = df_main[df_main['year'].isin(set(time_period))]

# Check if all countries still in the dataset
print("Number of countries after: ",len(df_main['country'].unique()) == len(eu_codes))

## Check if all years still in the dataset for every country
for c in df_main['country'].unique():
    for_this_c = df_main[df_main['country'] == c]
    all_years = (for_this_c['year'] == time_period).all()
    if not all_years:
        print("Not Consecutive years included for: ", c)
print("Rest have all years included")

df_main


In [ ]:
df_main.describe()

In [ ]:
df_main.info()

## Average DataFrame

In [ ]:
# Making a average DataFrame
df_average = df_main.groupby(["year"], as_index=False)
df_average = df_average.mean()
df_average


## Total DataFrame

In [ ]:
# Making total DataFrame
df_total = df_main.groupby(["year"], as_index=False)
df_total = df_total.sum()
df_total

In [ ]:
## Write all DataFrames to excel

writer = pd.ExcelWriter('data/main.xlsx', engine='xlsxwriter')

df_main.to_excel(writer, sheet_name = "main")
df_average.to_excel(writer, sheet_name = "average")
df_total.to_excel(writer, sheet_name = "total")
#critical last step
writer.save()

In [ ]:
s = "country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,co2_including_luc_per_capita,co2_including_luc_per_gdp,co2_including_luc_per_unit_energy,co2_per_capita,co2_per_gdp,co2_per_unit_energy,coal_co2,coal_co2_per_capita,consumption_co2,consumption_co2_per_capita,consumption_co2_per_gdp,cumulative_cement_co2,cumulative_co2,cumulative_co2_including_luc,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_luc_co2,cumulative_oil_co2,cumulative_other_co2,energy_per_capita,energy_per_gdp,flaring_co2,flaring_co2_per_capita,gas_co2,gas_co2_per_capita,ghg_excluding_lucf_per_capita,ghg_per_capita,land_use_change_co2,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,other_co2_per_capita,other_industry_co2,primary_energy_consumption,share_global_cement_co2,share_global_co2,share_global_co2_including_luc,share_global_coal_co2,share_global_cumulative_cement_co2,share_global_cumulative_co2,share_global_cumulative_co2_including_luc,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_luc_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share"

for i in s.split(","):
    print(i)